In [1]:
import pandas as pd

In [4]:
reviews = pd.read_csv('data/winemag-data-130k-v2.csv')

In [5]:
reviews.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [13]:
wines = reviews[['title','description']]
titles = reviews[['title']]

In [62]:
# Clean descriptions - remove certain punctuation
wines = wines.assign(clean_desc=wines.description.str.replace('[^ ,;\.\?\-—!…\w\d]', ''))

In [63]:
# Clean descriptions - remove multiple white spaces
wines = wines.assign(clean_desc=wines.clean_desc.str.replace('\s', ' '))

In [64]:
# Clean descriptions - remove multiple punctuation
wines = wines.assign(clean_desc=wines.clean_desc.str.replace('\.{3}', '…'))

In [65]:
# Clean descriptions - remove white spaces around commas and other punctuation

In [11]:
wines = wines.assign(words=wines.clean_desc.str.split(pat=' '))

AttributeError: 'DataFrame' object has no attribute 'clean_desc'

In [67]:
# wines.head()

In [27]:
def split_pairs(x):
    """Split a list into a list of 2-tuples."""
    output = []
    for i in range(0, len(x)):
        if i == 0:
            output.append(tuple(['_START', x[i]]))
        output.append(tuple(x[i:i+2]))
    return output

In [69]:
# sample = wines.iloc[0,3]
# [x for x in split_pairs(sample)]

In [70]:
wines = wines.assign(pairs=wines.apply(lambda row: split_pairs(row.words), axis=1))

In [71]:
words = wines['pairs'].apply(pd.Series).stack().to_frame(name='pairs')

In [72]:
punctuation = ['.', '?', '!']
words = words.assign(
    word_1=words['pairs'].apply(lambda x: '_START' if any([x[0].endswith(p) for p in punctuation]) else x[0]),
    word_2=words['pairs'].apply(lambda x: x[1] if len(x) > 1 else '_END')
)

In [73]:
# words.head()

In [74]:
words = words.reset_index()
words = words[['pairs', 'word_1', 'word_2']]

In [88]:
# Remove any empty words, filter out pairs that occur infrequently to make it faster
# len(wordcounts.index)
words = words[(words.word_1 != '') & (words.word_2 != '') & (words.word_2 != '_END')]
# wordcounts = wordcounts[wordcounts.n > 5]

In [76]:
wsample = words.sample(200)

In [90]:
w1 = words.groupby('word_1').agg('size').reset_index(name='w1_freq')
w2 = words.groupby(['word_1', 'word_2']).agg('size').reset_index(name='n')
wordcounts = w2.merge(w1, on='word_1', how='inner')

In [91]:
wordcounts = wordcounts.assign(weight=wordcounts.n/wordcounts.w1_freq)

In [99]:
wordcounts[wordcounts.word_1 == '_START'].sample(1, weights=wordcounts.weight).iloc[0,1]

'Its'

In [100]:
prev_word = '_START'
review = []
for i in range(20):
    prev_word = wordcounts[wordcounts.word_1 == prev_word].sample(1, weights=wordcounts.weight).iloc[0, 1]
    print(prev_word)

Its
a
blend
with
a
dusty
texture
full
bodied,
it
over
the
tannins
that
is
a
rich,
full-bodied
red.


ValueError: Invalid weights: weights sum to zero

In [ ]:
# Titles

In [22]:
# Clean titles - remove everything in parentheses
titles = titles.assign(clean_title=titles.title.str.replace('\(.*\)', ''))
titles = titles.assign(clean_title=titles.clean_title.str.strip())

In [26]:
titles = titles.assign(words=titles.clean_title.str.split(pat=' '))

In [28]:
titles = titles.assign(pairs=titles.apply(lambda row: split_pairs(row.words), axis=1))

,title,clean_title,words,pairs
0,Nicosia 2013 Vulkà Bianco (Etna),Nicosia 2013 Vulkà Bianco,"[Nicosia, 2013, Vulkà, Bianco]","[(_START, Nicosia), (Nicosia, 2013), (2013, Vu..."
1,Quinta dos Avidagos 2011 Avidagos Red (Douro),Quinta dos Avidagos 2011 Avidagos Red,"[Quinta, dos, Avidagos, 2011, Avidagos, Red]","[(_START, Quinta), (Quinta, dos), (dos, Avidag..."
2,Rainstorm 2013 Pinot Gris (Willamette Valley),Rainstorm 2013 Pinot Gris,"[Rainstorm, 2013, Pinot, Gris]","[(_START, Rainstorm), (Rainstorm, 2013), (2013..."
3,St. Julian 2013 Reserve Late Harvest Riesling ...,St. Julian 2013 Reserve Late Harvest Riesling,"[St., Julian, 2013, Reserve, Late, Harvest, Ri...","[(_START, St.), (St., Julian), (Julian, 2013),..."
4,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,"[Sweet, Cheeks, 2012, Vintner's, Reserve, Wild...","[(_START, Sweet), (Sweet, Cheeks), (Cheeks, 20..."


In [29]:
title_words = titles['pairs'].apply(pd.Series).stack().to_frame(name='pairs')

In [33]:
title_words = title_words.reset_index()
punctuation = ['.', '?', '!']
title_words = title_words.assign(
    word_1=title_words['pairs'].apply(lambda x: '_START' if any([x[0].endswith(p) for p in punctuation]) else x[0]),
    word_2=title_words['pairs'].apply(lambda x: x[1] if len(x) > 1 else '_END')
)
title_words = title_words[['word_1', 'word_2', 'pairs']]

In [37]:
title_words = title_words[(title_words.word_1 != '') & (title_words.word_2 != '') & (title_words.word_2 != '_END')]

In [38]:
title_words.head()

,word_1,word_2,pairs
0,_START,Nicosia,"(_START, Nicosia)"
1,Nicosia,2013,"(Nicosia, 2013)"
2,2013,Vulkà,"(2013, Vulkà)"
3,Vulkà,Bianco,"(Vulkà, Bianco)"
5,_START,Quinta,"(_START, Quinta)"


In [39]:
w1 = title_words.groupby('word_1').agg('size').reset_index(name='w1_freq')
w2 = title_words.groupby(['word_1', 'word_2']).agg('size').reset_index(name='n')
title_wordcounts = w2.merge(w1, on='word_1', how='inner')

In [40]:
title_wordcounts.head()

,word_1,word_2,n,w1_freq
0,#,Espino,1,2
1,#,Quillay,1,2
2,#1,Purisima,2,3
3,#1,Zinfandel,1,3
4,#10,Cabernet,1,7


In [41]:
title_wordcounts = title_wordcounts.assign(weight=title_wordcounts.n/title_wordcounts.w1_freq)

In [46]:
prev_word = '_START'
title = []
for i in range(6):
    prev_word = title_wordcounts[title_wordcounts.word_1 == prev_word].sample(1, weights=title_wordcounts.weight).iloc[0, 1]
    print(prev_word)

Longboard
2014
Kloof
Street
2006
Sauvignon


In [47]:
len(title_wordcounts.index)

166808

In [44]:
####### OLD #######

In [96]:
# Create a matrix of weights
# M = wordcounts[['word_1', 'word_2', 'weight']].pivot(index='word_2', columns='word_1', values='weight')
# M = M.fillna(0)

In [ ]:
M.iloc[:, 0].reset_index()[['word_2']].sample(1, weights=M['_START'])

In [ ]:
wordcounts.sort_values('n')

In [ ]:
# Everything below is not for the matrix approach

In [ ]:
wordcounts = wordcounts.sort_values('count', ascending=False)

In [ ]:
# Group by word_1, create a list of (word_2, count) tuples ordered by count
wordcounts = wordcounts.assign(next_words=wordcounts[['word_2', 'count']].values.tolist())

In [ ]:
starting_words = wordcounts[['word_1', 'next_words']].groupby('word_1').agg(lambda x: list(x))

In [ ]:
starting_words